Código del repo de la materia siu

La mejor opción es leer la descripción en el repo de la materia, si lo intento pegar, se termina viendo horrible.

In [7]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al azar''.
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

from sklearn.neighbors import NearestNeighbors

neighbors = NearestNeighbors(metric="euclidean")

In [8]:
class FlockAgent(Agent):
    def __init__(self, id, model, x, y, width, height):
        super().__init__(id, model)
        
        # Esto nos da una posición aleatoria en el espacio para el agente
        self.position = np.array((x, y), dtype = np.float64)
        
        # Esto nos da una velocidad aleatoria en el espacio para el agente
        vec = (np.random.rand(2) - 0.5) * 10
        self.velocity = np.array(vec, dtype = np.float64)
        
        # Esto nos da una aceleración aleatoria en el espacio para el agente
        vec = (np.random.rand(2) - 0.5) / 2
        self.acceleration = np.array(vec, dtype = np.float64)
        
        self.max_force = 0.3
        self.max_speed = 5
        self.witdh = width
        self.height = height   
        self.perception = 50
        
    # Esta definición hace que el agente se mueva
    def step(self):
        self.check_edges()
        self.check_width_neighbors()
    
        self.position = self.position + self.velocity
        self.velocity = self.velocity + self.acceleration
        
        if np.linalg.norm(self.velocity) > self.max_speed:
            self.velocity = self.velocity / (np.linalg.norm(self.velocity) * self.max_speed)
                
        #El np.float64 es para que no se convierta en un float32, esto es para que no haya problemas de precisión.
        self.acceleration = np.array((0, 0), dtype = np.float64)
                
    # Esta definicion hace que sea toroidal, es decir, que si se sale por un lado, aparece por el otro.
    def check_edges(self):
        if self.position.flatten()[0] > self.width:
            self.position[0] = 0
        elif self.position.flatten()[0] < 0:
            self.position[0] = self.width
            
        if self.position.flatten()[1] > self.height:
            self.position[1] = 0
        elif self.position.flatten()[1] < 0:
            self.position[1] = self.height    
            
    # Esta definición revisa los vecinos del agente
    def check_with_neighbors(self):
        alignment = self.align()
        cohesion = self.cohesion()
        separation = self.separation()
        
        self.acceleration += alignment
        self.acceleration += cohesion   
        self.acceleration += separation 
        
    #esta definición alinea al agente con sus vecinos
    def align(self):
        steering = np.array((0,0), dtype = np.float64)
        avg_vector = np.array((0,0), dtype = np.float64)
        
        result = neighbors.raious_neighbors([self.position],
                                            self.perception)[0]
        
        #Esto nos da el promedio de las velocidades de los agentes
        for idx in result:
            avg_vector += self.model.schedule.agents[idx].velocity
            
            
        total = len(result)
        if total > 0:
            avg_vector /= total
            #Ajustamos el vector para que esté dentro de los limites de velocidad. Una vez ajustado, se define el giro (steering) que va a tener
            avg_vector /= (avg_vector / np.linalg.norm(avg_vector)) * self.max_speed
            steering = avg_vector - self.velocity
            
        return steering
            
    